In [2]:
from numpy import log10, pi
from numpy.linalg import norm
import numpy as np
from scipy.constants import c
import scipy.optimize as opt
import fileinput
import time
import glob
import json
import datetime
import itertools
import pandas as pd
import matplotlib.pyplot as plt

f = 2.4e9

def _create_args(sensors, sigma):
    return (sensors, sigma)

def _get_args(args):
    """
        Returns elements from args created by create_args
        RETURN (sensors, sigma)
    """
    return args[0], args[1]

def _friis(sensor, x, y, Pt=1, z=2):
    return (Pt + 20 * log10(c / (4 * pi * f)) -
            10 * log10((x - sensor['x'])**2 + (y - sensor['y'])**2 + z**2))

def _loss(pos, *args):
    sensors, sigma = _get_args(args)
    return sum([(sensor['signal'] - _friis(sensor, pos[0], pos[1]))**2 / sigma**2 for sensor in sensors])

def read_data(ids=[]):
    start = time.time()
    data = {}
    files = glob.glob('/media/nickyz/Data/scriptie_data/armin/20170512_[5].csv')
    print(files)
    files.reverse()

    for line in fileinput.input(files=files):
        if fileinput.isfirstline():
            print(time.time() - start)
            print(fileinput.filename())
            start = time.time()
            continue

        splitted = line.split('\t')
        values = json.loads(splitted[0])

        try:
            sourceMac = values['sourceMac']
        except KeyError:
            sourceMac = values['sourcemac']

        if ids != [] and sourceMac not in ids:
            continue
        
        try:
            localMac = int(values['localMac'])
        except KeyError:
            localMac = int(values['localmac'])

        signal = int(values['signal'])

        if localMac:
            continue

        try:
            droneId = values['droneId']
        except KeyError:
            droneId = values['droneid']

        timestamp = splitted[2]
        orig_time = datetime.datetime.fromtimestamp(int(timestamp[0:-3])).strftime('%Y-%m-%d %H:%M:%S')
        timestamp = datetime.datetime.fromtimestamp(int(timestamp[0:-3])).strftime('%Y-%m-%d %H:%M')

        try:
            data[sourceMac].append([timestamp, droneId, signal, orig_time])
        except KeyError:
            data[sourceMac] = [[timestamp, droneId, signal, orig_time]]

    print(time.time() - start)

    return data

def read_toppers_data(ids=[]):
    """
        0 -> sourcemac
        1 -> sensorid
        2 -> typenr
        3 -> signal
        4 -> localmac
        5 -> seqnr
        6 -> retryflag
        7 -> subtypenr
        8 -> coordx
        9 -> coordy
        10 -> type
        11 -> source
        12 -> version
        13 -> measurementtimestamp
        14 -> processingtimestamp
        15 -> application
        16 -> date
    """
    start = time.time()
    data = {}
    files = glob.glob('/media/nickyz/Data/scriptie_data/u2/20170729_[2-4].csv')
    print(files)
    files.reverse()

    for line in fileinput.input(files=files):
        if fileinput.isfirstline():
            print(time.time() - start)
            print(fileinput.filename())
            start = time.time()
            continue

        splitted = line.split('\t')

        sourceMac = splitted[0]
        localMac = int(splitted[4])
        signal = int(splitted[3])

        if localMac:
            continue
        
        if ids != [] and sourceMac not in ids:
            continue
            
        droneId = splitted[1]

        timestamp = splitted[13]
        orig_time = datetime.datetime.fromtimestamp(int(timestamp[0:-3])).strftime('%Y-%m-%d %H:%M:%S')
        timestamp = datetime.datetime.fromtimestamp(int(timestamp[0:-3])).strftime('%Y-%m-%d %H:%M')

        try:
            data[sourceMac].append([timestamp, droneId, signal, orig_time])
        except KeyError:
            data[sourceMac] = [[timestamp, droneId, signal, orig_time]]

    print(time.time() - start)

    return data

def read_drone_data():
    drone_data = pd.read_csv('/media/nickyz/Data/scriptie_data/Huawei_routers_locations_at_Arena/sensors_arena_sensation.csv')

    return drone_data

def find_location(drone_id, drone_data):
    drone = drone_data.loc[drone_data['drone_id'] == drone_id]

    try:
        return drone.iloc[0]
    except IndexError:
        return None

def group_data(data, sort_index):
    sorted_data = sorted(data, key=lambda x: x[sort_index])

    return itertools.groupby(sorted_data, lambda x: x[sort_index])

In [3]:
data = read_data(["36f8ef8e-d6aa-4ac2-91f8-b6f2779a2974"])
drone_data = read_drone_data()

['/media/nickyz/Data/scriptie_data/armin/20170512_5.csv']
0.1135249137878418
/media/nickyz/Data/scriptie_data/armin/20170512_5.csv
33.049444913864136


In [17]:
time_grouped = group_data(data["36f8ef8e-d6aa-4ac2-91f8-b6f2779a2974"], 0)
sigma = 1

In [18]:
results = []
for time_group in time_grouped:
    sensors = []
    ids = []
    
    values = list(time_group[1])

    g_data = [(item[0], item[1], item[2], item[3]) for item in values]
    g_data = sorted(g_data, key=lambda x: -x[2])
    
    i = 0
    while(len(sensors) < 3 and i < len(g_data)):
        loc = find_location(g_data[i][1], drone_data)
        
        if loc is not None and g_data[i][1] not in ids:
            sensors.append({'signal': float(g_data[i][2]), 'x': loc['x_m'], 'y': loc['y_m']})
            ids.append(g_data[i][1])
        
        i += 1

    args = _create_args(sensors, sigma)
    res = opt.minimize(_loss, np.array([0.0, 0.0]), args=args)
    results.append(list(res.x) + [sensors])
    print(res.x)

[-110.05393188   66.9736305 ]
[ 28.02022915  79.94933498]
[ -10.25109423  105.00619088]
[  56.10908464  121.51168735]
[ 120.31047702   24.42898892]
[ 117.16010198  -11.92799305]
[ 111.23833383    8.43288088]
[ 51.06409688  61.78003056]
[ 23.36913634  95.39089239]
[  56.09386982  113.99836323]
[  50.46682178  112.6111703 ]
[  64.43194201  116.0537522 ]
[  72.77173089  118.10975844]
[  60.53947467  115.09393696]
[ 51.22740521  97.72413314]
[  41.09067871  123.10549833]
[  45.09816634  111.28791942]
[  11.07881974  136.01060797]
[  53.03022256  113.24317986]
[  55.01580135  113.73239158]
[ 40.49170813  89.85184946]
[  57.14523985  114.25737888]
[  50.46682178  112.6111703 ]
[  55.01580135  113.73239158]
[  51.45070828  112.85370776]
[  57.14523985  114.25737888]
[  59.42186762  114.81860357]
[  50.104749   112.5219477]
[  57.14523985  114.25737888]
[  59.42186762  114.81860357]
[  57.15795075  114.25999456]
[  59.52496053  114.84391954]
[  51.45070828  112.85370776]
[ 41.60253616  92.5444

In [21]:
xs = [row['x_m'] for i, row in drone_data.iterrows() if row['x_m'] < 1000]
ys = [row['y_m'] for i, row in drone_data.iterrows() if row['y_m'] < 1000]

for res in results[:1]:
    for loc in res[2]:
        plt.scatter(loc['x'], loc['y'], s=150, alpha=0.3)
    plt.scatter(xs, ys)
    plt.scatter(res[0], res[1], color='r')
    plt.show()

/usr/lib/python3/dist-packages/matplotlib/collections.py:549: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [15]:
loc_map = {
    'Lima': (5.82, 5.48),
    'Mike': (11.33, 9.43),
    'Kilo': (12.39, 6.77),
    'Oscar': (2.48, 7.36),
    'Alpha': (8.53, 2.16),
    'India': (2.18, 5.61),
    'November': (8.34, 4.13),
    'Hotel': (5.43, 4.71),
    'Romeo': (10.99, 5.94),
    'Quebec': (6.82, 9.78),
    'Papa': (9.9, 10.39)
}

def read_prepped_data():
    data = []
    prepped_data = pd.read_csv('~/CLS/Thesis/data/wifi-prepped-data.csv')
    
    for i, row in prepped_data.iterrows():
        orig_time = datetime.datetime.fromtimestamp(int(str(row['measurementTimestamp'])[0:-3])).strftime('%Y-%m-%d %H:%M:%S')
        timestamp = datetime.datetime.fromtimestamp(int(str(row['measurementTimestamp'])[0:-3])).strftime('%Y-%m-%d %H:%M')
        location = loc_map[row['droneId']]
        signal = row['signal']
        data.append([timestamp, location, signal, orig_time])
        
    return data

data = read_prepped_data()
time_grouped = group_data(data, 0)
sigma = 1

for time_group in time_grouped:
    sensors = []
    
    values = list(time_group[1])

    g_data = [(item[0], item[1], item[2], item[3]) for item in values]
    g_data = sorted(g_data, key=lambda x: -x[2])
    
    i = 0
    while(len(sensors) < 5 and i < len(g_data)):
        sensors.append({'signal': g_data[i][2], 'x': g_data[i][1][0], 'y': g_data[i][1][1]})
        
        i += 1

    args = _create_args(sensors, sigma)
    res = opt.minimize(_loss, np.array([0.0, 0.0]), args=args)
    print(res.x)

[ 9.10379423  9.11440336]
[ 10.95664567   2.42089255]
[ 18.41360723  -0.10353693]
[ 9.37452622  2.70114335]
[  9.87820309 -11.81822301]
[ 26.70897289 -21.63677506]
[ 10.9473204  -22.89952135]
[ 0.56075332  0.14199768]
[ 10.99173094 -23.36155929]
[-5.26291221 -1.3325787 ]
[ 25.66272994 -25.21678545]
[-4.94264495 -1.25158779]
[ 10.77094557 -21.07475348]
